
---

### Методология сбора данных

Для обучения модели оценки арендной стоимости в Москве используется выборка, сформированная на основе объявлений с сайта CIAN с помощью библиотеки `cianparser`. Цель — создать датасет, сбалансированный по ценовым сегментам, чтобы избежать переобучения модели на отдельных группах (например, слишком дешёвом или дорогом жилье) и обеспечить более реалистичное поведение модели на практике.

#### Структура выборки
- **Общий целевой объём:** ~16 000 объектов
- **Ожидаемое распределение:**
  - До 100 000 ₽: ~70%
  - 100 000–250 000 ₽: ~30%

#### Принципы сбалансированного сбора
Данные загружаются по заранее заданным ценовым диапазонам (бином) — от эконом-сегмента до средне-высокого. Для каждого бина задаётся квота, отражающая **реальное распределение предложений на рынке**, с учётом ограничения сайта (до 54 страниц на диапазон) и доступности объявлений.

> Такой подход позволяет получить выборку, близкую по структуре к текущему состоянию рынка, и избежать искажения метрик за счёт смещённого распределения. Модель, обученная на этих данных, будет лучше отражать реальные рыночные закономерности.

---

In [1]:
!pip install cianparser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 18.3 MB/s eta 0:00:00


In [ ]:
from typing import List, Tuple
from cianparser import CianParser
import pandas as pd
import time
import math

def parse_cian_data(
    location: str,
    price_ranges: List[Tuple[int, int]],
    total_target: int,
    avail_counts: dict
) -> pd.DataFrame:
    """
    Собирает объявления с CIAN по заданным ценовым диапазонам, распределяя дефицит.
    """
    parser = CianParser(location=location)
    seen_urls = set()
    all_data = []

    shares = {"low": 0.70, "mid": 0.30}
    low_bins  = price_ranges[:8]
    mid_bins  = price_ranges[8:]

    group_targets = {
        "low":  int(total_target * shares["low"]),
        "mid":  int(total_target * shares["mid"]),
    }

    desired = {}
    for name, bins in [("low", low_bins), ("mid", mid_bins)]:
        n_total = group_targets[name]
        base    = n_total // len(bins)
        rem     = n_total % len(bins)
        for i, rng in enumerate(bins):
            desired[rng] = base + (1 if i < rem else 0)

    MAX_PAGES = 54
    MAX_PER_BIN = MAX_PAGES * 25  # ≈1350
    quotas = {}
    total_assigned = 0
    for rng, q in desired.items():
        avail = avail_counts.get(rng, 0)
        q0 = min(q, avail, MAX_PER_BIN)
        quotas[rng] = q0
        total_assigned += q0

    deficit = total_target - total_assigned
    if deficit > 0:
        extras = {
            rng: min(avail_counts.get(rng, 0), MAX_PER_BIN) - quotas[rng]
            for rng in price_ranges
        }
        total_extra = sum(v for v in extras.values() if v > 0)
        if total_extra > 0:
            for rng, cap in extras.items():
                if cap <= 0:
                    continue
                add = math.floor(deficit * (cap / total_extra))
                add = min(add, cap)
                quotas[rng] += add
                deficit -= add
            for rng, cap in sorted(extras.items(), key=lambda x: x[1], reverse=True):
                if deficit <= 0:
                    break
                if cap > 0:
                    add = min(cap, deficit)
                    quotas[rng] += add
                    deficit -= add

    for (lo, hi), target in quotas.items():
        print(f"\nДиапазон {lo}–{hi}: целевая квота {target}")
        collected = 0
        page = 1
        while collected < target and page <= MAX_PAGES:
            data = parser.get_flats(
                deal_type="rent_long", rooms="all", with_saving_csv=False,
                additional_settings={"min_price": lo, "max_price": hi,
                                     "start_page": page, "end_page": page}
            )
            if not data:
                break
            new_items = [d for d in data if d["url"] not in seen_urls]
            if new_items:
                need = target - collected
                batch = new_items[:need]
                all_data.extend(batch)
                seen_urls.update(d["url"] for d in batch)
                collected += len(batch)
            page += 1
            time.sleep(2)
        print(f"  Собрано {collected}/{target}")

    df = pd.DataFrame(all_data)

    return df

In [ ]:
PRICE_RANGES = [
    (30001,40000),(40001,45000),(45001,50000),
    (50001,55000),(55001,60000),(60001,65000),(65001,70000),
    (70001,80000),(80001,90000),(90001,100000),
    (100001,120000),(120001,140000),(140001,180000),
    (180001,250000)
]

AVAIL_COUNTS = {
    (30001,40000):517, (40001,45000):750,
    (45001,50000):1250,(50001,55000):1250,(55001,60000):1500,
    (60001,65000):1300,(65001,70000):1350,(70001,80000):2200,
    (80001,90000):1650,(90001,100000):1350,(100001,120000):1700,
    (120001,140000):1150,(140001,180000):1350,(180001,250000):1350
}

TOTAL_TARGET = 16_000
LOCATION = "Москва"

data = parse_cian_data(
    location=LOCATION,
    price_ranges=PRICE_RANGES,
    total_target=TOTAL_TARGET,
    avail_counts=AVAIL_COUNTS
)


🔎 Диапазон 0–30000: целевая квота 113

                              Preparing to collect information from pages..
The page from which the collection of information begins: 
 https://cian.ru/cat.php?engine_version=2&p=1&with_neighbors=0&region=1&deal_type=rent&offer_type=flat&type=4&minprice=0&maxprice=30000

 1 | 1 page with list: [=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>] 100% | Count of all parsed: 28. Progress ratio: 100 %. Average price: 114 682 rub

The collection of information from the pages with list of offers is completed
Total number of parsed offers: 28. 

                              Preparing to collect information from pages..
The page from which the collection of information begins: 
 https://cian.ru/cat.php?engine_version=2&p=2&with_neighbors=0&region=1&deal_type=rent&offer_type=flat&type=4&minprice=0&maxprice=30000

 1 | 2 page with list: [=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>=>] 100% | Count of all parsed: 28. Progress ratio: 100 %.

In [ ]:
print(f"\nВсего в датасете: {len(data)} объявлений (цель {TOTAL_TARGET})")
data.to_csv(f"cian_rent_Moscow_balanced.csv", index=False, encoding="utf-8-sig")
print(f"Готово! Датасет сохранён как cian_rent_Moscow_balanced.csv")